In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import resource
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
# ------- Define modular methods for the task
def log_max_mem_usage():
    print(
        "Current all-time max memory: {} MB".format(
            resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000
        )
    )

sample_submission.csv
sample_submission.csv.zip
test.csv
test.csv.zip
train.csv
train.csv.zip



In [2]:
train_df = pd.read_csv('../input/train.csv')
train_df.fillna('zxzxzx zxzxzx', inplace=True) # For id: qid2 174364

test_df = pd.read_csv('../input/test.csv')
test_df.fillna('zxzxzx zxzxzx', inplace=True)

train_df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
train_df.shape

(404290, 6)

In [4]:
%%time

dataset_store = pd.HDFStore('../input/dataset.hdf', mode='w')
dataset_store.append('train_df', train_df)
dataset_store.append('test_df', test_df)
dataset_store.close()

CPU times: user 15.6 s, sys: 15.8 s, total: 31.4 s
Wall time: 5min 52s


In [5]:
%%time
dataset_store = pd.HDFStore('../input/dataset.hdf', mode='r')
dd = dataset_store.select('train_df', where=train_df.index[10:20])
dataset_store.close()
print dd

    id  qid1  qid2                                          question1  \
10  10    21    22  Method to find separation of slits using fresn...   
11  11    23    24        How do I read and find my YouTube comments?   
12  12    25    26               What can make Physics easy to learn?   
13  13    27    28        What was your first sexual experience like?   
14  14    29    30  What are the laws to change your status from a...   
15  15    31    32  What would a Trump presidency mean for current...   
16  16    33    34                       What does manipulation mean?   
17  17    35    36  Why do girls want to be friends with the guy t...   
18  18    37    38  Why are so many Quora users posting questions ...   
19  19    39    40  Which is the best digital marketing institutio...   

                                            question2  is_duplicate  
10  What are some of the things technicians can te...             0  
11             How can I see all my Youtube comments?   

In [6]:
# from joblib import Parallel, delayed

# def score_heuristic_batch(samp_index):
#     dataset_store = pd.HDFStore('../input/dataset.hdf', mode='r')
#     samp = dataset_store.select('train_df', where=samp_index)
#     dataset_store.close()

#     heuristics_scores = []

#     for row in samp.iterrows():
# #         if row[0] and row[0] % 10000 == 0:
# #             print(row[0])
#         heuristics_scores.append(score_row(row))

#     heuristics_scores = pd.DataFrame(heuristics_scores, index=samp.index)

#     return heuristics_scores


# def heuristic_score_parallel_interface(t_df):
#     return delayed(score_heuristic_batch)(t_df.index)


# def tfidf_score_parallel_interface(t_df):
#     return delayed(get_tfidf_features)(t_df)


# def parallel_scorer(samp, scorer_interface, batch, num_proc):
#     # Consumes 1.5G for batch=1000 and num_proc=4 for tfidf interface
#     # Use tfidf_features::batch=1000, heuristic_features::batch=20000
#     # scorer_interface::[heuristic_score_parallel_interface, tfidf_score_parallel_interface]
#     # Adjust batch depending on the interface used since the memory is dependent on the batch used.

#     with Parallel(n_jobs=num_proc) as parallel:
#         dataset = []
#         is_break = False
#         i = 0

#         while not is_break:
#             payload = []

#             for j in xrange(num_proc):
#                 t_df = samp[(i + j) * batch: (i + 1 + j) * batch]

#                 if t_df.empty:
#                     is_break = True
#                     continue

#                 payload.append(scorer_interface(t_df))
#             print((i + j) * batch)

#             if payload:
#                 results = parallel(payload)
#                 dataset.extend(results)
#                 i += num_proc

#     return pd.concat(dataset)


# def parallel_get_heuristic_scores(samp, batch=10000, num_proc=4):
#     return parallel_scorer(samp, heuristic_score_parallel_interface, batch, num_proc)


# def parallel_get_tfidf_scores(samp, batch=1000, num_proc=4):
#     return parallel_scorer(samp, tfidf_score_parallel_interface, batch, num_proc)

In [7]:
# %%time
# d = parallel_get_heuristic_scores(train_df.head(200000), batch=10000, num_proc=4)
# d

In [8]:
# d

In [9]:
log_max_mem_usage()

Current all-time max memory: 6660 MB


In [10]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion

unique_questions = pd.Series(pd.concat([train_df.question1, train_df.question2]).unique())
# combined_featurizers.fit(unique_questions)

char_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))  # featurizers[0][1]
word_tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2))  # featurizers[1][1]
char_tfidf.fit(unique_questions)
word_tfidf.fit(unique_questions)

log_max_mem_usage()

Current all-time max memory: 6660 MB
CPU times: user 2min 2s, sys: 2.69 s, total: 2min 4s
Wall time: 2min 7s


In [11]:
# %%time
# d = get_tfidf_features(train_df[:100000], batch=1000)

In [12]:
from nltk.corpus import stopwords
from collections import Counter
import re


stops = set(stopwords.words("english"))
num_pattern = re.compile('[0-9]+')
math_pattern = re.compile('\[math\](.*)\[\/math\]')

nums = '01234567890'


def get_tfidf_features(data_df, batch=1000):
    # data_df size should be about 40000 when used in parallel to observe effects of optimization.
    i = 0
#     print('Current data size in `get_tfidf_features`: {}'.format(data_df.shape[0]))

    word_dataset = np.array([])
    char_dataset = np.array([])

    while True:
        samp = data_df[i * batch: (i + 1) * batch]
        i += 1
#         print_batch = i * batch % 10000

#         if print_batch == 0:
#             print('Batch process in `get_tfidf_features`: {}'.format(i * batch))

        if samp.empty:
            break

        word_res = np.dot(
            word_tfidf.transform(samp.question1),
            word_tfidf.transform(samp.question2).T
        ).diagonal()
        
        char_res = np.dot(
            char_tfidf.transform(samp.question1),
            char_tfidf.transform(samp.question2).T
        ).diagonal()

        word_dataset = np.concatenate([word_dataset, word_res])
        char_dataset = np.concatenate([char_dataset, char_res])

    return pd.DataFrame(dict(wv=word_dataset, cv=char_dataset), index=data_df.index)


def get_heuristic_scores(q1, q2, ns_q1, ns_q2, swap):
#     n_q1 = {}
#     n_q2 = {}

#     for n in nums:
#         qc1 = q1.count(n)
#         qc2 = q2.count(n)
#         n_q1['q1_{}'.format(n)] = qc1
#         n_q2['q2_{}'.format(n)] = qc2

    if swap:
        q1 = ns_q1
        q2 = ns_q2

    exact_nums_q1 = num_pattern.findall(q1)
    exact_nums_q2 = num_pattern.findall(q2)
    
    math_q1 = math_pattern.findall(q1)
    math_q2 = math_pattern.findall(q2)

    num_exact_nums_match = len([n1 for n1 in exact_nums_q1 if n1 in exact_nums_q2])
    math_pattern_match = len([n1 for n1 in math_q1 if n1 in math_q2])
    
    is_q1_math = 1 * any(math_q1)
    is_q2_math = 1 * any(math_q2)
    is_both_math = is_q1_math * is_q2_math

#     qq2 = pd.Series(Counter([s for s in q1 if s.isupper()]))
#     qq1 = pd.Series(Counter([s for s in q2 if s.isupper()]))
    
#     sim_caps_rate = (qq1/qq2).mean()
#     num_caps_q1 = qq1.sum() 
#     num_caps_q2 = qq2.sum()

#     mean_caps_q1 = qq1.mean() 
#     mean_caps_q2 = qq2.mean()
    
    num_terms_q1 = len(q1.split())
    num_terms_q2 = len(q2.split())
    
    len_q1 = len(q1)
    len_q2 = len(q2)

    res = dict(
        num_exact_nums_match=num_exact_nums_match,
        math_pattern_match=math_pattern_match,
        is_q1_math=is_q1_math,
        is_q2_math=is_q2_math,
        is_both_math=is_both_math,
        length_diff=abs(len_q1 - len_q2),
        len_q1=len_q1,
        len_q2=len_q2,
        word_num_diff=abs(num_terms_q1 - num_terms_q2),
        num_terms_q1=num_terms_q1,
        num_terms_q2=num_terms_q2,
#         sim_caps_rate=sim_caps_rate,
#         mean_caps_q1=mean_caps_q1,
#         mean_caps_q2=mean_caps_q2,
#         num_caps_q1=num_caps_q1,
#         num_caps_q2=num_caps_q2,
    )
    
    # res.update(n_q1)
    # res.update(n_q2)
    
    return res


heuristics_feature_names = [
    'num_exact_nums_match',
    'math_pattern_match',
    'is_q1_math',
    'is_q2_math',
    'is_both_math',
    'length_diff',
    'len_q1',
    'len_q2',
    'word_num_diff',
    'num_terms_q1',
    'num_terms_q2',
]


def score_row(row, check_stops=False, swap=False):
    ix, row = row

    q1 = row.question1
    q2 = row.question2
    
    ns_q1 = [i for i in q1.lower().split() if i not in stops]
    ns_q2 = [i for i in q1.lower().split() if i not in stops]

    if not all([ns_q1, ns_q2]) and check_stops:
        if ix % 5 == 0:
            # 39405
            print('here! {}'.format(ix))

        return {i: np.nan for i in heuristics_feature_names}
    
    ns_q1 = ' '.join(ns_q1)
    ns_q2 = ' '.join(ns_q2)

    return get_heuristic_scores(q1, q2, ns_q1, ns_q2, swap=swap)

In [13]:
ix =  203
row = train_df.ix[ix]
q1 = row.question1
q2 = row.question2
print q1
print q2
print set(q1.lower().split()).difference(stops)
print set(q2.lower().split()).difference(stops)
print row.is_duplicate
print score_row((ix, row))

Why do people hate Hillary Clinton?
What are the reasons that people dislike Hillary Clinton?
set(['hillary', 'hate', 'clinton?', 'people'])
set(['reasons', 'hillary', 'dislike', 'clinton?', 'people'])
1
{'is_q2_math': 0, 'math_pattern_match': 0, 'is_q1_math': 0, 'length_diff': 22, 'num_terms_q1': 6, 'num_terms_q2': 9, 'is_both_math': 0, 'num_exact_nums_match': 0, 'word_num_diff': 3, 'len_q1': 35, 'len_q2': 57}


In [14]:
# %%time
# ds = []
# # samp = train_df[404000:]
# samp = train_df.head(200000)  # [30000:50000]
# for row in samp.iterrows():
#     ds.append(score_row(row))

# ds = pd.DataFrame(ds, index=samp.index)
# #ds

In [44]:
# %%time
from sklearn.model_selection import train_test_split
import multiprocessing as mp
import time
from joblib import Parallel, delayed


def score_heuristic_batch(samp_index, is_train):
    dataset_store = pd.HDFStore('../input/dataset.hdf', mode='r')
    if is_train:
        samp = dataset_store.select('train_df', where=samp_index)
    else:
        samp = dataset_store.select('test_df', where=samp_index)
    dataset_store.close()

    heuristics_scores = []

    for row in samp.iterrows():
#         if row[0] and row[0] % 10000 == 0:
#             print(row[0])

        heuristics_scores.append(score_row(row))

    heuristics_scores = pd.DataFrame(heuristics_scores, index=samp.index)

    return heuristics_scores


def heuristic_score_parallel_interface(t_df, is_train):
    return delayed(score_heuristic_batch)(t_df.index, is_train)


def tfidf_score_parallel_interface(t_df, is_train):
    return delayed(get_tfidf_features)(t_df)


def parallel_scorer(samp, scorer_interface, is_train, batch, num_proc):
    # Consumes 1.5G for batch=1000 and num_proc=4 for tfidf interface
    # Use tfidf_features::batch=10000, heuristic_features::batch=20000
    # scorer_interface::[heuristic_score_parallel_interface, tfidf_score_parallel_interface]
    # Adjust batch depending on the interface used since the memory is dependent on the batch used.

    with Parallel(n_jobs=num_proc) as parallel:
        dataset = []
        is_break = False
        i = 0

        while not is_break:
            payload = []

            for j in xrange(num_proc):
                t_df = samp[(i + j) * batch: (i + 1 + j) * batch]

                if t_df.empty:
                    is_break = True
                    continue

                payload.append(scorer_interface(t_df, is_train))
            print('Current batch in main thread: {}'.format((i + j) * batch))

            if payload:
                results = parallel(payload)
                dataset.extend(results)
                i += num_proc

    return pd.concat(dataset)


def parallel_get_heuristic_scores(samp, is_train, batch=10000, num_proc=4):
    return parallel_scorer(samp, heuristic_score_parallel_interface, is_train, batch, num_proc)


def parallel_get_tfidf_scores(samp, is_train, batch=40000, num_proc=4):
    # The batch size for the size of the dataset should be large to maximize effect of parallelization.
    # The batch here is different from the batch used in the method `get_tfidf_features`
    return parallel_scorer(samp, tfidf_score_parallel_interface, is_train, batch, num_proc)


NUM_PROC = max(1, mp.cpu_count() - 1)

def parallel_get_features(samp, is_train, tfidf_sample_batch=40000, heuristics_sample_batch=10000):
    start_time = time.time()

    tfidf_features = parallel_get_tfidf_scores(samp, is_train, batch=tfidf_sample_batch, num_proc=NUM_PROC)
    print('Finished computing tfidf features after {} seconds.'.format((time.time() - start_time)))

    heuristics_scores = parallel_get_heuristic_scores(samp, is_train, batch=heuristics_sample_batch, num_proc=NUM_PROC)
    print('Finished computing heuristic features after {} seconds.'.format((time.time() - start_time)))

    features = pd.concat([tfidf_features, heuristics_scores], axis=1)
    return features


def get_features(samp):
    start_time = time.time()
    
    tfidf_features = get_tfidf_features(samp)
    print('Finished computing tfidf features after {} seconds.'.format((time.time() - start_time)))

    heuristics_scores = []

    for row in samp.iterrows():
        if row[0] and row[0] % 10000 == 0:
            print(row[0])
  
        heuristics_scores.append(score_row(row))

    heuristics_scores = pd.DataFrame(heuristics_scores, index=samp.index)
    
    features = pd.concat([tfidf_features, heuristics_scores], axis=1)
    
    return features


# samp = train_df

# X_train = get_features(samp)
# y_train = samp.is_duplicate

# log_max_mem_usage()

In [16]:
# %%time
# parallel_get_tfidf_scores(train_df.head(60000), batch=30000, num_proc=NUM_PROC)
# get_tfidf_features(train_df.head(60000), batch=1000)  # , num_proc=NUM_PROC)

In [27]:
# heuristics_scores = []
# samp = train_df[404285: 404293]
# for row in samp.iterrows():
#     if row[0] and row[0] % 10000 == 0:
#         print(row[0])

#     heuristics_scores.append(score_row(row))
# heuristics_scores

In [43]:
ix = 410000
batch = 30000
samp = test_df[ix: ix + batch]
print(samp.index)
dataset_store = pd.HDFStore('../input/dataset.hdf', mode='r')
# sp = dataset_store.select('train_df', where=samp.index)
sp = dataset_store.select('test_df', where=samp.index)
dataset_store.close()
# sp

RangeIndex(start=410000, stop=440000, step=1)


In [29]:
%%time
X_train = parallel_get_features(train_df, is_train=True)
y_train = train_df.is_duplicate

log_max_mem_usage()

Current batch in main thread: 80000
Current batch in main thread: 200000
Current batch in main thread: 320000
Current batch in main thread: 440000
Finished computing tfidf features after 147.836364031 seconds.
Current batch in main thread: 20000
Current batch in main thread: 50000
Current batch in main thread: 80000
Current batch in main thread: 110000
Current batch in main thread: 140000
Current batch in main thread: 170000
Current batch in main thread: 200000
Current batch in main thread: 230000
Current batch in main thread: 260000
Current batch in main thread: 290000
Current batch in main thread: 320000
Current batch in main thread: 350000
Current batch in main thread: 380000
Current batch in main thread: 410000
Finished computing heuristic features after 222.511899948 seconds.
Current all-time max memory: 6660 MB
CPU times: user 1.84 s, sys: 480 ms, total: 2.32 s
Wall time: 3min 42s


In [30]:
# %%time
# X_train = get_features(samp)
# y_train = samp.is_duplicate

# log_max_mem_usage()

In [31]:
train_df.shape

(404290, 6)

In [32]:
# %%time

pos_train = X_train[y_train == 1]
neg_train = X_train[y_train == 0]

# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((1.0 * len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    print(scale)
    
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1

neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

X_train = pd.concat([pos_train, neg_train])
y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()

del pos_train, neg_train


X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=1029)

log_max_mem_usage()

1.23756274561
0
Current all-time max memory: 6660 MB


In [33]:
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss, roc_auc_score, make_scorer


scaler = StandardScaler()

char_lm_model = LogisticRegression(C=100)
word_lm_model = LogisticRegression(C=1)
length_diff_lm_model = LogisticRegression(C=1)
word_num_diff_lm_model = LogisticRegression(C=1)

rf_model = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, min_samples_split=3, n_jobs=NUM_PROC)
lm_model = LogisticRegression()

def log_loss_scorer(model, X, y):
    return log_loss(y, model.predict_proba(X))

def fit_models(X, y):
    rf_model.fit(X_train, y_train)
    char_lm_model.fit(X_train.cv.values.reshape(-1, 1), y_train)
    word_lm_model.fit(X_train.wv.values.reshape(-1, 1), y_train)
    length_diff_lm_model.fit(X_train.length_diff.values.reshape(-1, 1), y_train)
    word_num_diff_lm_model.fit(X_train.word_num_diff.values.reshape(-1, 1), y_train)

def predict(X):
#     weights = dict(zip(X.columns, rf_model.feature_importances_))
    char_pred = char_lm_model.predict_proba(X.cv.values.reshape(-1, 1))[:, 1] # * weights['cv']
    word_pred = word_lm_model.predict_proba(X.wv.values.reshape(-1, 1))[:, 1] # * weights['wv']
    length_diff_pred = length_diff_lm_model.predict_proba(X.length_diff.values.reshape(-1, 1))[:, 1] # * weights['length_diff']
    word_num_diff_pred = word_num_diff_lm_model.predict_proba(X.word_num_diff.values.reshape(-1, 1))[:, 1] # * weights['word_num_diff']
    rf_pred = rf_model.predict_proba(X)[:, 1]

    return [char_pred, word_pred, length_diff_pred, word_num_diff_pred, rf_pred]

In [34]:
# %%time

lm_model.fit(X_train, y_train)

print(log_loss_scorer(lm_model, X_valid, y_valid))

0.410091383085


In [35]:
# %%time

rf_model.fit(X_train, y_train)

print(log_loss_scorer(rf_model, X_valid, y_valid))

0.286679794441


In [36]:
# %%time
# fit_models(X_train, y_train)
# cross_val_score(rf_model, X_train, y_train, scoring=log_loss_scorer)

In [37]:
print(roc_auc_score(y_valid, rf_model.predict_proba(X_valid)[:, 1]))

0.925450508732


In [15]:
# print(log_loss(y_test, np.mean(predict(X_test), axis=0)))
# print(roc_auc_score(y_test, np.mean(predict(X_test), axis=0)))
# print(roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))
# print(sum((y_test - (1 * (rf_model.predict_proba(X_test)[:, 1] < 0.5))) != 0))

In [16]:
# for i, j in sorted(list(zip(X_test.columns, rf_model.feature_importances_)), key=lambda x: x[1], reverse=True):
#     print i, j

In [17]:
# pd.concat([features, samp.is_duplicate], axis=1).groupby('is_duplicate').mean().T

In [23]:
test_df.fillna('zxzxzx zxzxzx', inplace=True)

In [38]:
1.0 * test_df.shape[0] / train_df.shape[0]

5.802260753419575

In [45]:
%%time
X_test = parallel_get_features(test_df, is_train=False)

log_max_mem_usage()

Current batch in main thread: 80000
Current batch in main thread: 200000
Current batch in main thread: 320000
Current batch in main thread: 440000
Current batch in main thread: 560000
Current batch in main thread: 680000
Current batch in main thread: 800000
Current batch in main thread: 920000
Current batch in main thread: 1040000
Current batch in main thread: 1160000
Current batch in main thread: 1280000
Current batch in main thread: 1400000
Current batch in main thread: 1520000
Current batch in main thread: 1640000
Current batch in main thread: 1760000
Current batch in main thread: 1880000
Current batch in main thread: 2000000
Current batch in main thread: 2120000
Current batch in main thread: 2240000
Current batch in main thread: 2360000
Finished computing tfidf features after 715.204516172 seconds.
Current batch in main thread: 20000
Current batch in main thread: 50000
Current batch in main thread: 80000
Current batch in main thread: 110000
Current batch in main thread: 140000
Curr

In [46]:
%%time
X_test.to_hdf('kaggle-quora', 'test-features-X_test')

/home/avsolatorio/canopy/lib/python2.7/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'test-features-X_test'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


CPU times: user 176 ms, sys: 200 ms, total: 376 ms
Wall time: 784 ms


In [47]:
test_id = test_df.test_id

# del(test_df)
# del(train_df)

In [48]:
import gc
gc.collect()

215

In [49]:
# %%time
lm_sub = pd.DataFrame()

lm_sub['test_id'] = test_id
lm_sub['is_duplicate'] = lm_model.predict_proba(X_test)[:, 1]

lm_sub.to_csv('lm_submission.csv', index=False)
lm_sub.head()

,test_id,is_duplicate
0,0,0.065448
1,1,0.161394
2,2,0.213189
3,3,0.043126
4,4,0.037919


In [50]:
# %%time
rf_sub = pd.DataFrame()

rf_sub['test_id'] = test_id
rf_sub['is_duplicate'] = rf_model.predict_proba(X_test)[:, 1]

rf_sub.to_csv('rf_submission.csv', index=False)
rf_sub.head()

,test_id,is_duplicate
0,0,0.048285
1,1,0.365262
2,2,0.149548
3,3,0.005025
4,4,0.173174


In [ ]:
from IPython.display import FileLink
test_df = pd.read_csv('../input/test.csv')

scores_data = []
samp = train_df

for row in samp.iterrows():
    if row[0] % 5000 == 0:
        print(row[0])

    scores_data.append(score_row(row))

X = pd.DataFrame(scores_data)
is_duplicate_test = np.mean(predict(X), axis=0)

log_max_mem_usage()

In [ ]:
del(train_df)